
# Surprise 협업필터링 기반 추천시스템

## 데이터프레임 로드

In [1]:
import pandas as pd
df = pd.read_csv('추천시스템감성분석용_완.csv')
df

,숙소ID,color,style,view,별점,작성자,동반유형,리뷰내용,작성일,감성별점
0,1000003240,white,minimal,ocean,1,니제*,NaN,"바다 안보임, 테라스 금연, 식수 없음, 객실 청결상태 안좋음, 드라이도 사용하기 ...",2022.10.02,1
1,1000003240,white,minimal,ocean,2,브뤼셀*****,NaN,난방을 안해주셔서 춥게잤네요,2022.10.24,2
2,1000003240,white,minimal,ocean,3,영월2박3***,아이와 함께,방 깨끗합니다. 주변 산책하며 바닷가도 가까워 좋아요.\n아쉬운 점은 방에 욱풍이 ...,2024.11.27,4
3,1000003240,white,minimal,ocean,3,작은그*****,NaN,막 별로였던 건 아니지만 생각보다 세련된 느낌의 펜션은 아니었어요.\n그래도 원래 ...,2020.02.13,5
4,1000003240,white,minimal,ocean,4,초코렛이리*****,친구와 함께,바다와는 쪼금 거리가 있었지만 사장님이 친절했어요!\n주방용품은 다 잘 되어있었는데...,1일 전,5
...,...,...,...,...,...,...,...,...,...,...
643838,3020735,white,minimal,other,5,으니062*,NaN,머무는 동안 즐거웠습니다 ♡,2023.10.02,3
643839,3020735,white,minimal,other,5,바스크조약***,NaN,입실하기 전부터 강화도 날씨 안내 겸 웰컴 문자를 보내주시고 편하고 재밌게 잘 쉬고...,2023.10.02,5
643840,3020735,white,minimal,other,5,sis79**,NaN,좋습니다ㅎ,2023.09.30,5
643841,3020735,white,minimal,other,5,알파고사랑*******,NaN,침대 매트리스에 긴머리카락 수두룩~~\n사장님이 넘 친절하셔서 좋앗다.\n그이상...,2023.09.29,5


- 5점이 너무 높은 분포를 차지하고있음

In [2]:
df['감성별점'].value_counts()   

감성별점
5    286478
1    141404
4    134349
3     59304
2     22308
Name: count, dtype: int64

In [3]:
# 작성자컬럼 앞뒤 공백 제거
df['작성자'] = df['작성자'].str.strip()

- 데이터에 문제가 있는지 확인

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643843 entries, 0 to 643842
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   숙소ID    643843 non-null  int64 
 1   color   643843 non-null  object
 2   style   643843 non-null  object
 3   view    643843 non-null  object
 4   별점      643843 non-null  int64 
 5   작성자     643843 non-null  object
 6   동반유형    214177 non-null  object
 7   리뷰내용    643618 non-null  object
 8   작성일     643843 non-null  object
 9   감성별점    643843 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 49.1+ MB


In [6]:
df['숙소ID'] = df['숙소ID'].astype(str)

In [7]:
df.describe()

,별점,감성별점
count,643843.000000,643843.000000
mean,3.981944,3.624669
std,1.199806,1.584742
min,1.000000,1.000000
25%,3.000000,2.000000
50%,4.000000,4.000000
75%,5.000000,5.000000
max,5.000000,5.000000


### 외부 데이터프레임 사용시 주의점
- 데이터 정제후 필요없는열은 제거.
- 순서를 꼭 지켜주세요. 사용자-아이템-평점
- surprise의 reader함수로 평점의 범위를 기록합니다.  

In [8]:
from surprise import Reader
reader = Reader(rating_scale=(1, 5))

In [9]:
from surprise import Dataset, Reader

# 평점의 범위를 정의합니다 (예: 1점~5점)
# 제공된 데이터의 감성별점은 1,2,3으로 보입니다.
reader = Reader(rating_scale=(1, 5))

# 데이터프레임에서 필요한 열을 선택하여 Surprise 데이터셋으로 로드합니다
surprise_data = Dataset.load_from_df(df[['작성자', '숙소ID', '감성별점']], reader)

## 데이터 나누기
- 이제, 본격적으로 모델훈련 후 추천시스템을 만듭니다.

In [10]:
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

#data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=42)

## 알고리즘 선택과 학습

### 예측함수 test(), predict()
- test() 모든 검증 데이터에 대한 평점 예측
- predict() 하나의 사용자와 아이템에 대한 예측

In [11]:
# 알고리즘 선택 및 학습 (SVD 사용)
algo = SVD()
algo.fit(trainset)

# 테스트셋에 대한 예측 및 성능 평가
pred = algo.test(testset) # 예측
rmse = accuracy.rmse(pred) # 평가

RMSE: 1.4517


In [12]:
type(pred)

list

In [13]:
len(pred)

128769

In [14]:
pred[:5]

[Prediction(uid='가랑비********', iid='1000101009', r_ui=5.0, est=2.916804791071599, details={'was_impossible': False}),
 Prediction(uid='뚝섬샴****', iid='3011635', r_ui=5.0, est=3.7441887204992295, details={'was_impossible': False}),
 Prediction(uid='소풍스며들*****', iid='3009340', r_ui=1.0, est=3.1580957965698273, details={'was_impossible': False}),
 Prediction(uid='도깨비********', iid='1000103843', r_ui=4.0, est=4.187985370179964, details={'was_impossible': False}),
 Prediction(uid='조각구름알******', iid='10054622', r_ui=4.0, est=3.8178922169658707, details={'was_impossible': False})]

* 리스트 요소
  - uid:사용자 ("어느 사용자를 위한 예측인지")
  - iid:아이템id
  - r_ui:사용자 "실제" 평점
  - est:모델이 구한 예측평점

## 특정 사용자와 아이템의 평점 예측

In [15]:
df

,숙소ID,color,style,view,별점,작성자,동반유형,리뷰내용,작성일,감성별점
0,1000003240,white,minimal,ocean,1,니제*,NaN,"바다 안보임, 테라스 금연, 식수 없음, 객실 청결상태 안좋음, 드라이도 사용하기 ...",2022.10.02,1
1,1000003240,white,minimal,ocean,2,브뤼셀*****,NaN,난방을 안해주셔서 춥게잤네요,2022.10.24,2
2,1000003240,white,minimal,ocean,3,영월2박3***,아이와 함께,방 깨끗합니다. 주변 산책하며 바닷가도 가까워 좋아요.\n아쉬운 점은 방에 욱풍이 ...,2024.11.27,4
3,1000003240,white,minimal,ocean,3,작은그*****,NaN,막 별로였던 건 아니지만 생각보다 세련된 느낌의 펜션은 아니었어요.\n그래도 원래 ...,2020.02.13,5
4,1000003240,white,minimal,ocean,4,초코렛이리*****,친구와 함께,바다와는 쪼금 거리가 있었지만 사장님이 친절했어요!\n주방용품은 다 잘 되어있었는데...,1일 전,5
...,...,...,...,...,...,...,...,...,...,...
643838,3020735,white,minimal,other,5,으니062*,NaN,머무는 동안 즐거웠습니다 ♡,2023.10.02,3
643839,3020735,white,minimal,other,5,바스크조약***,NaN,입실하기 전부터 강화도 날씨 안내 겸 웰컴 문자를 보내주시고 편하고 재밌게 잘 쉬고...,2023.10.02,5
643840,3020735,white,minimal,other,5,sis79**,NaN,좋습니다ㅎ,2023.09.30,5
643841,3020735,white,minimal,other,5,알파고사랑*******,NaN,침대 매트리스에 긴머리카락 수두룩~~\n사장님이 넘 친절하셔서 좋앗다.\n그이상...,2023.09.29,5


In [16]:
# uid와 iid는 문자열 형태이므로 문자열로 넣어줌
uid = '부산낡은배***' #유저id
iid = '1020479' #위 유저가 본적없는 영화 id
test = algo.predict(uid, iid)
print(test)

user: 부산낡은배***   item: 1020479    r_ui = None   est = 3.28   {'was_impossible': False}


## 교차검증과 하이퍼파라미터 튜닝

In [17]:
from surprise.model_selection import cross_validate
cross_validate(algo, surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4551  1.4534  1.4549  1.4483  1.4542  1.4532  0.0025  
MAE (testset)     1.1790  1.1789  1.1788  1.1743  1.1800  1.1782  0.0020  
Fit time          4.07    4.17    4.36    4.45    4.38    4.29    0.14    
Test time         0.60    0.59    0.37    0.36    0.64    0.51    0.12    


{'test_rmse': array([1.45511102, 1.45342121, 1.45486371, 1.44826708, 1.45422763]),
 'test_mae': array([1.17901685, 1.17885635, 1.17879481, 1.17428175, 1.17997587]),
 'fit_time': (4.071720838546753,
  4.170668125152588,
  4.361380100250244,
  4.4482245445251465,
  4.378214597702026),
 'test_time': (0.5969216823577881,
  0.5911753177642822,
  0.37001657485961914,
  0.35722827911376953,
  0.6440277099609375)}

In [18]:
from surprise import Dataset, SVD, KNNBasic, BaselineOnly
# SVD 알고리즘 교차 검증
print("=== SVD ===")
svd = SVD()  # 기본 파라미터로 SVD 알고리즘 사용
svd_results = cross_validate(svd, surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# # KNNBasic 알고리즘 교차 검증
# print("\n=== KNNBasic ===")
# knn = KNNBasic()  # 기본 파라미터로 KNNBasic 알고리즘 사용
# knn_results = cross_validate(knn, surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# BaselineOnly 알고리즘 교차 검증
print("\n=== BaselineOnly ===")
base = BaselineOnly()  # 기본 파라미터로 BaselineOnly 알고리즘 사용
base_results = cross_validate(base, surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# 결과 요약
print("\n=== Summary ===")
print(f"SVD mean RMSE: {sum(svd_results['test_rmse'])/5:.4f}, mean MAE: {sum(svd_results['test_mae'])/5:.4f}")
#print(f"KNNBasic mean RMSE: {sum(knn_results['test_rmse'])/5:.4f}, mean MAE: {sum(knn_results['test_mae'])/5:.4f}")
print(f"BaselineOnly mean RMSE: {sum(base_results['test_rmse'])/5:.4f}, mean MAE: {sum(base_results['test_mae'])/5:.4f}")

=== SVD ===


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4536  1.4526  1.4541  1.4501  1.4549  1.4531  0.0017  
MAE (testset)     1.1789  1.1770  1.1792  1.1764  1.1788  1.1781  0.0011  
Fit time          3.96    3.87    4.33    4.35    4.01    4.10    0.20    
Test time         0.63    0.68    0.37    0.68    0.36    0.54    0.14    

=== BaselineOnly ===
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4470  1.4546  1.4538  1.4532  1.4544  1.4526  0.0029  
MAE (testset)     1.2192  1.2265  1.2267  1.2249  1.2257  1.2246  0.0028  
Fit time          0.93    0.92    0.96    1.00    1.00    0.96    0.03    
Test time         0.63    0.60 

- 최적의 하이퍼파라미터 찾기

In [19]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors':[50, 100, 200]}
                #에포크 학습 반복.        svd 잠재요인
# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(surprise_data)
# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.4516613503181688
{'n_epochs': 20, 'n_factors': 50}


- 최적의 파라미터로 재학습

In [20]:
best_p = gs.best_params['rmse']
trainset = surprise_data.build_full_trainset()
algo = SVD(n_epochs=best_p['n_epochs'], n_factors=best_p['n_factors'])
algo.fit(trainset)

In [21]:
# GridSearchCV 결과에서 얻은 최적의 파라미터로 SVD 모델을 생성
best_params = gs.best_params['rmse']
best_svd = SVD(n_epochs=best_params['n_epochs'], n_factors=best_params['n_factors'])

# 전체 학습 데이터셋을 사용하여 최종 모델을 학습
best_svd.fit(trainset)

# === 이 부분을 피클로 저장해야 합니다 ===
# ...existing code...
import joblib
joblib.dump(best_svd, 'best_svd_model.joblib')
# ...existing code...

['best_svd_model.joblib']

In [38]:
# 예시: 특정 사용자 ID ('나폴리트레****'와 같은 실제 작성자 ID)
user_id = '바스크조약***'

# 전체 데이터셋에서 해당 사용자가 이미 평점을 매긴 숙소 목록을 가져옵니다.
rated_accommodations = df.loc[df['작성자'] == user_id, '숙소ID'].tolist()

# 전체 숙소 목록을 가져옵니다.
all_accommodations = df['숙소ID'].unique()

# 아직 평점을 매기지 않은 숙소 목록을 필터링합니다.
unrated_accommodations = [item for item in all_accommodations if item not in rated_accommodations]

# 필터링된 숙소들에 대한 예상 평점을 계산합니다.
predictions = []
for accommodation_id in unrated_accommodations:
    predicted_rating = best_svd.predict(user_id, accommodation_id)
    predictions.append(predicted_rating)

In [39]:
# ...existing code...
predictions = []
for accommodation_id in unrated_accommodations:
    predicted_rating = best_svd.predict(user_id, accommodation_id)
    predictions.append(predicted_rating)

# 예측 평점을 기준으로 내림차순 정렬하여 상위 10개 추천
predictions.sort(key=lambda x: x.est, reverse=True)
top_10_recommendations = predictions[:10]

print(f"\n{user_id} 님에게 추천하는 숙소 TOP 10:")
for i, pred in enumerate(top_10_recommendations):
    print(f"{i+1}위: 숙소ID {pred.iid} - 예상 평점: {pred.est}")
# ...existing code...


바스크조약*** 님에게 추천하는 숙소 TOP 10:
1위: 숙소ID 1000092412 - 예상 평점: 5
2위: 숙소ID 1000094308 - 예상 평점: 5
3위: 숙소ID 1000100216 - 예상 평점: 5
4위: 숙소ID 1000101965 - 예상 평점: 5
5위: 숙소ID 1000109519 - 예상 평점: 5
6위: 숙소ID 1000111415 - 예상 평점: 5
7위: 숙소ID 10041674 - 예상 평점: 5
8위: 숙소ID 10044478 - 예상 평점: 5
9위: 숙소ID 10052014 - 예상 평점: 5
10위: 숙소ID 3002268 - 예상 평점: 5


In [24]:
# 예측 평점을 기준으로 내림차순 정렬하여 상위 10개 추천
predictions.sort(key=lambda x: x.est, reverse=True)
top_150_recommendations = predictions[:150]

print(f"\n{user_id} 님에게 추천하는 숙소 TOP 150:")
for i, pred in enumerate(top_150_recommendations):
    print(f"{i+1}위: 숙소ID {pred.iid} - 예상 평점: {pred.est}")


나폴리트레**** 님에게 추천하는 숙소 TOP 150:
1위: 숙소ID 3013225 - 예상 평점: 5
2위: 숙소ID 1000094308 - 예상 평점: 4.9201140536926
3위: 숙소ID 1000111415 - 예상 평점: 4.895832626208463
4위: 숙소ID 1000100216 - 예상 평점: 4.889916219319052
5위: 숙소ID 3002268 - 예상 평점: 4.885659963209895
6위: 숙소ID 1000109516 - 예상 평점: 4.876221804880176
7위: 숙소ID 3010529 - 예상 평점: 4.874419341268908
8위: 숙소ID 3011813 - 예상 평점: 4.861573486473325
9위: 숙소ID 3010266 - 예상 평점: 4.8560255870079905
10위: 숙소ID 3016367 - 예상 평점: 4.855693776450955
11위: 숙소ID 1000110965 - 예상 평점: 4.846066870694903
12위: 숙소ID 1000087511 - 예상 평점: 4.845768392407097
13위: 숙소ID 10039349 - 예상 평점: 4.8456242430713905
14위: 숙소ID 1000109921 - 예상 평점: 4.8422480419913825
15위: 숙소ID 1000101965 - 예상 평점: 4.840891787307781
16위: 숙소ID 10044607 - 예상 평점: 4.836742755395974
17위: 숙소ID 3013096 - 예상 평점: 4.828314223717406
18위: 숙소ID 3008405 - 예상 평점: 4.819866969627826
19위: 숙소ID 1000104986 - 예상 평점: 4.819326744725826
20위: 숙소ID 3016494 - 예상 평점: 4.8179767153099835
21위: 숙소ID 10042877 - 예상 평점: 4.816415917053639
22위: 숙소ID 301413

# 추천 시스템 모델 완성

In [161]:
# # ...existing code...
# import joblib

# # 저장된 모델 로드
# loaded_svd = joblib.load('best_svd_model.joblib')

# # 추천 예시
# user_id = '나폴리트레****'
# rated_accommodations = df.loc[df['작성자'] == user_id, '숙소ID'].tolist()
# all_accommodations = df['숙소ID'].unique()
# unrated_accommodations = [item for item in all_accommodations if item not in rated_accommodations]

# predictions = []
# for accommodation_id in unrated_accommodations:
#     predicted_rating = loaded_svd.predict(str(user_id), str(accommodation_id))
#     predictions.append(predicted_rating)

# predictions.sort(key=lambda x: x.est, reverse=True)
# top_10_recommendations = predictions[:10]

# print(f"\n{user_id} 님에게 추천하는 숙소 TOP 10:")
# for i, pred in enumerate(top_10_recommendations):
#     print(f"{i+1}위: 숙소ID {pred.iid} - 예상 평점: {pred.est}")
# # ...existing code...

In [28]:
import joblib

# 저장된 모델 로드
loaded_svd = joblib.load('best_svd_model.joblib')

# 추천 예시
user_id = df.iloc[200]['작성자']
rated_accommodations = df.loc[df['작성자'] == user_id, '숙소ID'].tolist()
all_accommodations = df['숙소ID'].unique()
unrated_accommodations = [item for item in all_accommodations if item not in rated_accommodations]

predictions = []
for accommodation_id in unrated_accommodations:
    predicted_rating = loaded_svd.predict(str(user_id), str(accommodation_id))
    predictions.append(predicted_rating)

predictions.sort(key=lambda x: x.est, reverse=True)
top_10_recommendations = predictions[:10]

print(f"\n{user_id} 님에게 추천하는 숙소 TOP 10:")
for i, pred in enumerate(top_10_recommendations):
    print(f"{i+1}위: 숙소ID {pred.iid} - 예상 평점: {pred.est}")


아가* 님에게 추천하는 숙소 TOP 10:
1위: 숙소ID 1000090694 - 예상 평점: 5
2위: 숙소ID 1000091713 - 예상 평점: 5
3위: 숙소ID 1000092412 - 예상 평점: 5
4위: 숙소ID 1000094308 - 예상 평점: 5
5위: 숙소ID 1000099537 - 예상 평점: 5
6위: 숙소ID 1000100216 - 예상 평점: 5
7위: 숙소ID 1000100665 - 예상 평점: 5
8위: 숙소ID 1000104896 - 예상 평점: 5
9위: 숙소ID 1000108134 - 예상 평점: 5
10위: 숙소ID 1000108458 - 예상 평점: 5


In [32]:
import joblib

# 저장된 모델 로드
loaded_svd = joblib.load('best_svd_model.joblib')

# 추천 예시
user_id = df.iloc[710]['작성자']
rated_accommodations = df.loc[df['작성자'] == user_id, '숙소ID'].tolist()
all_accommodations = df['숙소ID'].unique()
unrated_accommodations = [item for item in all_accommodations if item not in rated_accommodations]

predictions = []
for accommodation_id in unrated_accommodations:
    predicted_rating = loaded_svd.predict(str(user_id), str(accommodation_id))
    predictions.append(predicted_rating)

predictions.sort(key=lambda x: x.est, reverse=True)
top_10_recommendations = predictions[:10]

print(f"\n{user_id} 님에게 추천하는 숙소 TOP 10:")
for i, pred in enumerate(top_10_recommendations):
    print(f"{i+1}위: 숙소ID {pred.iid} - 예상 평점: {pred.est}")


윤준영* 님에게 추천하는 숙소 TOP 10:
1위: 숙소ID 1000092412 - 예상 평점: 5
2위: 숙소ID 1000094308 - 예상 평점: 5
3위: 숙소ID 1000100216 - 예상 평점: 5
4위: 숙소ID 1000100687 - 예상 평점: 5
5위: 숙소ID 1000109424 - 예상 평점: 5
6위: 숙소ID 1000109519 - 예상 평점: 5
7위: 숙소ID 1000109921 - 예상 평점: 5
8위: 숙소ID 1000111546 - 예상 평점: 5
9위: 숙소ID 10044607 - 예상 평점: 5
10위: 숙소ID 10051362 - 예상 평점: 5


# 모델 함수로 정리

In [192]:
import joblib

# 저장된 모델 로드
loaded_svd = joblib.load('best_svd_model.joblib')

def recommend_top_n(df, loaded_svd, user_index, n=10):

    user_id = df.iloc[user_index]['작성자']
    rated_accommodations = df.loc[df['작성자'] == user_id, '숙소ID'].tolist()
    all_accommodations = df['숙소ID'].unique()
    unrated_accommodations = [item for item in all_accommodations if item not in rated_accommodations]

    predictions = []
    for accommodation_id in unrated_accommodations:
        predicted_rating = loaded_svd.predict(str(user_id), str(accommodation_id))
        predictions.append(predicted_rating)

    predictions.sort(key=lambda x: x.est, reverse=True)
    top_n_recommendations = predictions[:n]

    print(f"\n{user_id} 님에게 추천하는 숙소 TOP {n}:")
    for i, pred in enumerate(top_n_recommendations):
        print(f"{i+1}위: 숙소ID {pred.iid} - 예상 평점: {pred.est}")

recommend_top_n(df, loaded_svd, user_index=256, n=10)


나무향******** 님에게 추천하는 숙소 TOP 10:
1위: 숙소ID 3017527 - 예상 평점: 4.907564233834231
2위: 숙소ID 1000090614 - 예상 평점: 4.891138351931032
3위: 숙소ID 3013225 - 예상 평점: 4.850819335141007
4위: 숙소ID 3011067 - 예상 평점: 4.8420596902835165
5위: 숙소ID 10046906 - 예상 평점: 4.779202021303834
6위: 숙소ID 1000112905 - 예상 평점: 4.763144298533348
7위: 숙소ID 3001685 - 예상 평점: 4.731474240797359
8위: 숙소ID 3012477 - 예상 평점: 4.722746901596379
9위: 숙소ID 10043186 - 예상 평점: 4.695583490987961
10위: 숙소ID 1014382 - 예상 평점: 4.689111485683483


In [186]:
# 사용 예시
recommend_top_n(df, loaded_svd, user_index=1, n=10)


브뤼셀***** 님에게 추천하는 숙소 TOP 10:
1위: 숙소ID 10048293 - 예상 평점: 4.775669327845827
2위: 숙소ID 1000109940 - 예상 평점: 4.771617285277465
3위: 숙소ID 3010529 - 예상 평점: 4.706556947695621
4위: 숙소ID 1000109424 - 예상 평점: 4.704361625083738
5위: 숙소ID 1000109116 - 예상 평점: 4.694499236515587
6위: 숙소ID 1000105195 - 예상 평점: 4.691318261509213
7위: 숙소ID 1000109516 - 예상 평점: 4.665621137309538
8위: 숙소ID 1000109591 - 예상 평점: 4.663392016988628
9위: 숙소ID 1000090694 - 예상 평점: 4.660880221425258
10위: 숙소ID 1000104309 - 예상 평점: 4.651471050360749
